In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from musicautobot.imports import *
from musicautobot.numpy_encode import *
from musicautobot.config import *
from musicautobot.vocab import *
from musicautobot.music_transformer.all import *
from musicautobot.utils.midifile import *
from musicautobot.utils.file_processing import process_all

## Preprocessing Large Dataset

This Notebook is specifically for preprocessing and encoding a large dataset. It uses multi-processing and handles encoding errors

***

**Note** This is the same training same code as `Train.ipynb`. Only thing that is different is a more robust pre-processing step

In [3]:
base_path = Path('../../data')

# Location of your midi files
midi_path = base_path/'midi/examples'
# Location of preprocessed numpy files
numpy_path = base_path/'numpy/lm'

# Location of models and cached dataset
data_path = base_path/'cached'
data_save_name = 'reddit_musicitem_data_save.pkl'

[p.mkdir(parents=True, exist_ok=True) for p in [midi_path, numpy_path, data_path]];

## 1. Gather midi dataset

Make sure all your midi data is in `musicautobot/data/midi` directory

Here's a pretty good dataset with lots of midi data:  
https://www.reddit.com/r/datasets/comments/3akhxy/the_largest_midi_collection_on_the_internet/

1. Download the folder and unzip it to `data/midi`

2. Rename `130000_Pop_...` to `reddit`

## 2. Create dataset from MIDI files

In [4]:
# num_tracks = [1, 2] # number of tracks to support
cutoff = 5 # max instruments
min_variation = 3 # minimum number of different midi notes played
# max_dur = 128

In [5]:
midi_files = get_files(midi_path, '.mid', recurse=True); len(midi_files)

31

In [6]:
def process_metadata(midi_file):
    # Get outfile and check if it exists
    out_file = numpy_path/midi_file.relative_to(midi_path).with_suffix('.npy')
    out_file.parent.mkdir(parents=True, exist_ok=True)
    if out_file.exists(): return
    
    npenc = transform_midi(midi_file)
    if npenc is not None: np.save(out_file, npenc)

In [7]:
def transform_midi(midi_file, debug=False):
    input_path = midi_file
    
    # Part 1: Filter out midi tracks (drums, repetitive instruments, etc.)
    try: 
#         if duet_only and num_piano_tracks(input_path) not in [1, 2]: return None
        input_file = compress_midi_file(input_path, min_variation=min_variation, cutoff=cutoff) # remove non note tracks and standardize instruments
        if input_file is None: return None
    except Exception as e:
        if debug: raise e
        if 'badly form' in str(e): return None # ignore badly formatted midi errors
        if 'out of range' in str(e): return None # ignore badly formatted midi errors
        print('Error parsing midi', input_path, e)
        return None
        
    # Part 2. Compress rests and long notes
    stream = file2stream(input_file) # 1.
    try:
        chordarr = stream2chordarr(stream) # 2. max_dur = quarter_len * sample_freq (4). 128 = 8 bars
    except Exception as e:
        if debug: raise e
        print('Could not encode to chordarr:', input_path, e)
        print(traceback.format_exc())
        return None
    
    # Part 3. Compress song rests - Don't want songs with really long pauses 
    # (this happens because we filter out midi tracks).
    chord_trim = trim_chordarr_rests(chordarr)
    chord_short = shorten_chordarr_rests(chord_trim)
    delta_trim = chord_trim.shape[0] - chord_short.shape[0]
#     if delta_trim > 500: 
#         print(f'Removed {delta_trim} rests from {input_path}. Skipping song')
#         return None
    chordarr = chord_short
    
    # Part 3. Chord array to numpy
    npenc = chordarr2npenc(chordarr)
    if not is_valid_npenc(npenc, input_path=input_path):
        return None
    
    return npenc

In [8]:
# # sanity check
import random
for r in random.sample(midi_files, 10):
#     transform_midi(r, debug=True)
    process_metadata(r)

In [9]:
def timeout_func(data, seconds):
    print("Timeout:", seconds, data.get('midi'))

In [10]:
processed = process_all(process_metadata, midi_files, timeout=120, timeout_func=timeout_func)

In [11]:
def create_databunch(files, data_save_name, path=data_path):
    save_file = path/data_save_name
    if save_file.exists():
        data = load_data(path, data_save_name)
    else:
        save_file.parent.mkdir(exist_ok=True, parents=True)
        vocab = MusicVocab.create()
        processors = [OpenNPFileProcessor(), MusicItemProcessor()]

        data = MusicDataBunch.from_files(files, path, processors=processors, encode_position=True)
        data.save(data_save_name)
    return data

## Create Data Bunch

In [12]:
numpy_files = get_files(numpy_path, extensions='.npy', recurse=True); len(numpy_files)

31

In [16]:
all_data = create_databunch(numpy_files, data_save_name=data_save_name); all_data

/home/ashaw/anaconda3/envs/musicautobot_v1/lib/python3.8/site-packages/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(a, dtype=dtype, **kwargs)


MusicDataBunch;

Train: LabelList (28 items)
x: MusicItemList

MusicItem - (304,)
xxbos xxpad n76 d6 n52 d8 n50 d8 n45 d8...,
MusicItem - (282,)
xxbos xxpad n69 d2 n52 d16 n48 d16 n45 d16...,
MusicItem - (134,)
xxbos xxpad n57 d2 n52 d16 n48 d16 n45 d16...,
MusicItem - (634,)
xxbos xxpad n62 d2 n57 d8 n53 d8 n50 d8...,
MusicItem - (560,)
xxbos xxpad n69 d4 n52 d16 n49 d16 n45 d16...
y: LMLabelList
,,,,
Path: ../../data/cached;

Valid: LabelList (3 items)
x: MusicItemList

MusicItem - (876,)
xxbos xxpad n67 d1 n55 d3 n50 d3 n48 d3...,
MusicItem - (312,)
xxbos xxpad n55 d16 n52 d16 n48 d16 xxsep d4...,
MusicItem - (352,)
xxbos xxpad xxsep d10 n57 d2 xxsep d2 n59 d2...
y: LMLabelList
,,
Path: ../../data/cached;

Test: None

## 3. Load Model

In [17]:
batch_size = 1
encode_position = True
dl_tfms = [batch_position_tfm] if encode_position else []
data = load_data(data_path, data_save_name, bs=batch_size, encode_position=encode_position, dl_tfms=dl_tfms)

In [18]:
config = default_config()
config['encode_position'] = encode_position
learn = music_model_learner(data, config=config.copy())

## 4. Train

In [ ]:
learn.fit_one_cycle(4)

In [ ]:
learn.save('example')

## 5. Predict

---
See [Generate.ipynb](Generate.ipynb) to use a pretrained model and generate better predictions

---

In [ ]:
midi_file = Path('data/midi/notebook_examples/single_bar_example.mid'); midi_file
item = MusicItem.from_file(midi_file, data.vocab);

In [ ]:
item.show()

Here's what the seed sounds like:

### Start Predictions:

In [ ]:
pred, full = learn.predict(item, n_words=100)

Prediction

In [ ]:
# Prediction
pred.show()

In [ ]:
pred.play()